In [1]:
from src_clean.dataloader.dataloader import DataLoader
from src_clean.preprocessor.prepro_new import PreprocessorTwo
import pandas as pd

In [2]:
train_df, test_df = DataLoader.load_train_test_dfs()
#train_df = PreprocessorTwo.preprocess(train_df, exclude_cols_to_drop=['srch_id', 'prop_id', 'position'])
train_df = PreprocessorTwo.remove_outliers(train_df)
train_df = PreprocessorTwo.relevance(train_df)
#train_df = PreprocessorTwo.prepro_comp_columns(train_df)
train_df = PreprocessorTwo.add_price_difference_srch_id(train_df)

#test_df = PreprocessorTwo.preprocess(test_df, exclude_cols_to_drop=['prop_id'])
#test_df = PreprocessorTwo.prepro_comp_columns(test_df)
test_df = PreprocessorTwo.add_price_difference_srch_id(test_df)
train_df, test_df = PreprocessorTwo.calculate_statistics(train_df, test_df)
train_df, test_df = PreprocessorTwo.mean_pos_per_prop_id(train_df, test_df)
#train_df, test_df = PreprocessorTwo.click_and_booking_prob(train_df, test_df)

train_df = PreprocessorTwo.extract_booking_month(train_df)
test_df = PreprocessorTwo.extract_booking_month(test_df)


In [4]:
test_df['position']

KeyError: 'position'

In [19]:
train_df, test_df = PreprocessorTwo.add_price_difference_prop_id(train_df, test_df)

In [20]:
train_df = PreprocessorTwo.add_price_order(train_df)
test_df = PreprocessorTwo.add_price_order(test_df)

In [3]:
train_df = pd.read_parquet(f'parquets/train_df_preprocessed_no_normalization_outliers_removed.parquet')
test_df = pd.read_parquet(f'parquets/test_df_preprocessed_no_normalization_outliers_removed.parquet')

In [4]:
train_df = PreprocessorTwo.visitor_price_diff(train_df)
train_df = PreprocessorTwo.visitor_starrating_diff(train_df)
test_df = PreprocessorTwo.visitor_price_diff(test_df)
test_df = PreprocessorTwo.visitor_starrating_diff(test_df)

In [ ]:
train_df = PreprocessorTwo.feature_normalization(train_df)
test_df = PreprocessorTwo.feature_normalization(test_df)

In [ ]:
# train_df = PreprocessorTwo.impute_competitors(train_df) nope
# test_df = PreprocessorTwo.impute_competitors(test_df) nope

train_df = PreprocessorTwo.drop_cols(train_df, exclude_cols_to_drop=['srch_id', 'prop_id'])
test_df = PreprocessorTwo.drop_cols(test_df)

In [ ]:
train_df.columns = [str(col) for col in train_df.columns]
test_df.columns = [str(col) for col in test_df.columns]

In [ ]:
# To Parquets
train_df.to_parquet('train_df_preprocessed_with_normalization_outliers_removed.parquet')
test_df.to_parquet('test_df_preprocessed_with_normalization_outliers_removed.parquet')

In [14]:
train_df = PreprocessorTwo.drop_cols(train_df, exclude_cols_to_drop=['srch_id', 'prop_id'])
test_df = PreprocessorTwo.drop_cols(test_df)

In [15]:
from src_clean.ranker.ranker import Ranker
from src_clean.score.ndcg import calculate_ndcg
import lightgbm
import numpy as np
X_train, X_valid, y_train, y_valid = DataLoader.split_df_into_train_and_val_batches(train_df, validation_size=0.1)
group_train = [group.shape[0] for group in X_train]
group_val = [group.shape[0] for group in X_valid]

In [16]:
params = {'num_iterations': 1000,
          'learning_rate': 0.047311784907414445,
          'max_depth': 500,
          'num_leaves': 272,
          'min_data_in_leaf': 9433,
          }

model = lightgbm.LGBMRanker(
    objective="lambdarank",
    metric="ndcg",
    early_stopping_round=300,
    **params
)
model.fit(
    X=np.vstack(X_train),
    y=np.hstack(y_train),
    group=group_train,
    eval_set=[(np.vstack(X_valid), np.hstack(y_valid))],
    eval_group=[group_val],
    eval_at=5,
    verbose=10
)
pred = model.predict(test_df.to_numpy())
ranker = Ranker()
ranking_file_path = ranker.make_ranking_from_prediction(
    pred, model_name="LightGBM"
)
calculate_ndcg(ranking_file_path)

[LightGBM] [Warning] min_data_in_leaf is set=9433, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=9433
[10]	valid_0's ndcg@5: 0.388368
[20]	valid_0's ndcg@5: 0.39502
[30]	valid_0's ndcg@5: 0.399395
[40]	valid_0's ndcg@5: 0.401699
[50]	valid_0's ndcg@5: 0.404035
[60]	valid_0's ndcg@5: 0.404694
[70]	valid_0's ndcg@5: 0.406087
[80]	valid_0's ndcg@5: 0.407542
[90]	valid_0's ndcg@5: 0.408521
[100]	valid_0's ndcg@5: 0.409794
[110]	valid_0's ndcg@5: 0.409861
[120]	valid_0's ndcg@5: 0.411615
[130]	valid_0's ndcg@5: 0.412119
[140]	valid_0's ndcg@5: 0.413878
[150]	valid_0's ndcg@5: 0.414847
[160]	valid_0's ndcg@5: 0.415525
[170]	valid_0's ndcg@5: 0.416823
[180]	valid_0's ndcg@5: 0.417517
[190]	valid_0's ndcg@5: 0.417844
[200]	valid_0's ndcg@5: 0.418229
[210]	valid_0's ndcg@5: 0.418541
[220]	valid_0's ndcg@5: 0.41934
[230]	valid_0's ndcg@5: 0.419416
[240]	valid_0's ndcg@5: 0.420048
[250]	valid_0's ndcg@5: 0.420196
[260]	valid_0's ndcg@5: 0.420862
[270]	valid_0's ndcg@5: 0.4

0.4232805827561173

In [9]:
ranking_file_path

'/Users/robinbux/Desktop/VU/Period5/DataMining/DMT_2022/Assignment_2/rankings/LightGBM/ranking_22_05_2022-35_16.csv'

In [ ]:
from joblib import dump, load
dump(model, 'lambda_mart.joblib')